In [1]:
# %pip install scikit-surprise # pickle

In [1]:
from surprise import accuracy, Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [2]:
import pandas as pd
import pathlib
DATA_DIR = pathlib.Path().resolve().parent / "data"
print(DATA_DIR)
DATA_DIR.exists()

C:\Users\paule\Desktop\Netflix-Clone\src\data


True

In [3]:
dataset = DATA_DIR / 'ratings_small.csv'
dataset.exists()

True

In [4]:
df = pd.read_csv(dataset)
df['rating'].dropna(inplace=True)
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
df.rating.max(), df.rating.min()

(5.0, 0.5)

In [9]:
# Setting surprise
# Adding rating scale
reader = Reader(rating_scale=(0.5, 5))

# Loading the dataframe whit 'userId', 'movieId', 'rating' coloms
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [10]:
# Declaring the algoritm
algo = SVD(verbose=True, n_epochs=20)

# Cross validate will compare the algoritm whit the data across RMSE function and MAE function
cross_validate(algo, data, measures=['RMSE', "MAE"], cv=4, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

{'test_rmse': array([0.89540823, 0.89760063, 0.90806832, 0.89783456]),
 'test_mae': array([0.68945877, 0.69293155, 0.69871321, 0.68940415]),
 'fit_time': (1.1546688079833984,
  1.0791113376617432,
  0.9958007335662842,
  0.9611310958862305),
 'test_time': (0.251873254776001,
  0.1800367832183838,
  0.18880200386047363,
  0.13486242294311523)}

In [11]:
# Training

trainset = data.build_full_trainset()
algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [14]:
# Accuracy

# Grab the trestset and some prediction from that testset
testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)
# accuracy.mae(predictions, verbose=True)

RMSE: 0.6436


0.6435888519053693

In [17]:
sample_row = df.sample(n=1)
userId = sample_row['userId'].values[0]
movieId = sample_row['movieId'].values[0]
print(userId, movieId)

599 71033


In [18]:
# Getting the estimated rating based on the user and movie id
pred = algo.predict(uid=userId, iid=movieId)
pred.est

3.318980858780768

In [19]:
import pickle

In [20]:
algo_data = {"model": algo}
with open('model.pkl', 'wb') as f:
    pickle.dump(algo_data, f)

In [21]:
model_algo= None
with open('model.pkl', 'rb') as f:
    model_data_loaded = pickle.load(f)
    model_algo = model_data_loaded.get('model')

In [22]:
model_algo.predict(uid=userId, iid=movieId).est

3.318980858780768

In [23]:
sample_rows = df.sample(n=10).to_dict('records')
for row in sample_rows:
    userId = row['userId']
    movieId = row['movieId']
    pred = model_algo.predict(uid=userId, iid=movieId).est
    print(userId, movieId, pred)

596 2706 3.6921048632112794
373 2390 3.177367049737415
130 1086 3.7435009764844285
63 3897 3.8319745035164687
415 10 3.8039183810950856
661 2352 3.545325679266506
15 53972 2.733854797621082
607 589 3.854806698736473
295 1084 4.101516191419939
99 2565 3.0632845425732325
